## Capstone Project (Week 3)

#### Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Scraping The Data

In [2]:
html_doc=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc.text, 'lxml')

### Creating a DataFrame

In [3]:
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    # Header
    if(index==0):
        columns=section
    else:
        data.append(section)
        
canada_df = pd.DataFrame(data = data, columns = columns)

### Cleaning The Data More And Making It more suitable for further Analysis

In [5]:
canada_df = canada_df[canada_df['Borough']!= 'Not assigned']
canada_df["Neighbourhood"] = canada_df.groupby("Postcode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))

#remove duplicates
canada_df = canada_df.drop_duplicates()

#update index to be postcode if it isn't already
#if(canada_df.index.name != 'Postcode'):
#   canada_df = canada_df.set_index('Postcode')

canada_df['Neighbourhood'].replace('Not assigned', canada_df['Borough'], inplace = True)
canada_df
neighbourhoods = canada_df
neighbourhoods = neighbourhoods.reset_index()
neighbourhoods = neighbourhoods.drop(columns = ['index'])
neighbourhoods.head(12)

Postcode           Borough  \
0       M3A        North York   
1       M4A        North York   
2       M5A  Downtown Toronto   
3       M6A        North York   
4       M7A      Queen's Park   
5       M9A         Etobicoke   
6       M1B       Scarborough   
7       M3B        North York   
8       M4B         East York   
9       M5B  Downtown Toronto   
10      M6B        North York   
11      M9B         Etobicoke   

                                        Neighbourhood  
0                                           Parkwoods  
1                                    Victoria Village  
2                           Harbourfront, Regent Park  
3                    Lawrence Heights, Lawrence Manor  
4                                        Queen's Park  
5                                    Islington Avenue  
6                                      Rouge, Malvern  
7                                     Don Mills North  
8                     Woodbine Gardens, Parkview Hill  
9                            Ryerson, Garden District  
10                                          Glencairn  
11  Cloverdale, Islington, Martin Grove, Princess ...

### Getting the Latitude and Longitudes

In [6]:
# Loading Latitude and Longitudes csv file 
lat_lon = pd.read_csv("https://cocl.us/Geospatial_data")

In [7]:
latitude=[]
longitude=[]
c=-1
for x in range(1, 104):
    c+=1
    for i,postal_code in enumerate(lat_lon['Postal Code']):
        p = neighbourhoods.iloc[c, 0]
        if (p == postal_code):
            latitude.append(lat_lon.iloc[i, 1])
            longitude.append(lat_lon.iloc[i, 2])

neighbourhoods['Latitude'] = latitude
neighbourhoods['Longitude'] = longitude
    
neighbourhoods

Postcode           Borough  \
0        M3A        North York   
1        M4A        North York   
2        M5A  Downtown Toronto   
3        M6A        North York   
4        M7A      Queen's Park   
5        M9A         Etobicoke   
6        M1B       Scarborough   
7        M3B        North York   
8        M4B         East York   
9        M5B  Downtown Toronto   
10       M6B        North York   
11       M9B         Etobicoke   
12       M1C       Scarborough   
13       M3C        North York   
14       M4C         East York   
15       M5C  Downtown Toronto   
16       M6C              York   
17       M9C         Etobicoke   
18       M1E       Scarborough   
19       M4E      East Toronto   
20       M5E  Downtown Toronto   
21       M6E              York   
22       M1G       Scarborough   
23       M4G         East York   
24       M5G  Downtown Toronto   
25       M6G  Downtown Toronto   
26       M1H       Scarborough   
27       M2H        North York   
28       M3H        North York   
29       M4H         East York   
30       M5H  Downtown Toronto   
31       M6H      West Toronto   
32       M1J       Scarborough   
33       M2J        North York   
34       M3J        North York   
35       M4J         East York   
36       M5J  Downtown Toronto   
37       M6J      West Toronto   
38       M1K       Scarborough   
39       M2K        North York   
40       M3K        North York   
41       M4K      East Toronto   
42       M5K  Downtown Toronto   
43       M6K      West Toronto   
44       M1L       Scarborough   
45       M2L        North York   
46       M3L        North York   
47       M4L      East Toronto   
48       M5L  Downtown Toronto   
49       M6L        North York   
50       M9L        North York   
51       M1M       Scarborough   
52       M2M        North York   
53       M3M        North York   
54       M4M      East Toronto   
55       M5M        North York   
56       M6M              York   
57       M9M        North York   
58       M1N       Scarborough   
59       M2N        North York   
60       M3N        North York   
61       M4N   Central Toronto   
62       M5N   Central Toronto   
63       M6N              York   
64       M9N              York   
65       M1P       Scarborough   
66       M2P        North York   
67       M4P   Central Toronto   
68       M5P   Central Toronto   
69       M6P      West Toronto   
70       M9P         Etobicoke   
71       M1R       Scarborough   
72       M2R        North York   
73       M4R   Central Toronto   
74       M5R   Central Toronto   
75       M6R      West Toronto   
76       M7R       Mississauga   
77       M9R         Etobicoke   
78       M1S       Scarborough   
79       M4S   Central Toronto   
80       M5S  Downtown Toronto   
81       M6S      West Toronto   
82       M1T       Scarborough   
83       M4T   Central Toronto   
84       M5T  Downtown Toronto   
85       M1V       Scarborough   
86       M4V   Central Toronto   
87       M5V  Downtown Toronto   
88       M8V         Etobicoke   
89       M9V         Etobicoke   
90       M1W       Scarborough   
91       M4W  Downtown Toronto   
92       M5W  Downtown Toronto   
93       M8W         Etobicoke   
94       M9W         Etobicoke   
95       M1X       Scarborough   
96       M4X  Downtown Toronto   
97       M5X  Downtown Toronto   
98       M8X         Etobicoke   
99       M4Y  Downtown Toronto   
100      M7Y      East Toronto   
101      M8Y         Etobicoke   
102      M8Z         Etobicoke   

                                         Neighbourhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656  
1                                     Victoria Village  43.725882 -79.315572  
2                            Harbourfront, Regent Park  43.654260 -79.360636  
3                     Lawrence Heights, Lawrence Manor  43.718518 -79.464763  
4                                         Queen's Park  43.662301 -79.38